In [14]:

import pandas as pd
import re
import scipy.io as spio
import numpy as np
import statistics
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2
import os

# Function to load MATLAB data

In [15]:
def loadmat(filename):
    def _check_keys(d):
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [16]:
def calculate_z_scores(dataframe, column_name):
    # Calculate the mean and standard deviation of the specified column
    mean = dataframe[column_name].mean()
    std = dataframe[column_name].std()
    
    # Calculate the Z-scores for the specified column
    z_scores = (dataframe[column_name] - mean) / std
    
    # Create a new DataFrame with the Z-scores added as a new column
    result_df = dataframe.copy()
    result_df['z_score_' + column_name] = z_scores
    
    return result_df

In [17]:
def plot_divided_dfs(divided_dfs, x_column, y_column, numeric_part, y_axis_name, min_value=None, max_value=None):
    import pandas as pd
    import plotly.graph_objs as go

    # Create a figure
    fig = go.Figure()

    # Define a time offset for each subset
    time_offset = 0

    # Iterate over the divided DataFrames
    for i, df in enumerate(divided_dfs):
        # Create a new filtered DataFrame based on the specified range
        filtered_df = df.copy()
        if min_value is not None:
            filtered_df = filtered_df[filtered_df[y_column] >= min_value]
        if max_value is not None:
            filtered_df = filtered_df[filtered_df[y_column] <= max_value]

        # Create a scatter trace for the current subset with the adjusted x-values
        trace = go.Scatter(
            x=filtered_df[x_column] + time_offset,
            y=filtered_df[y_column],
            mode='lines',
            name=f'Movie {i + 1}'
        )

        # Add the trace to the figure
        fig.add_trace(trace)

        # Update the time offset for the next subset
        time_offset += max(filtered_df[x_column])  # Use the maximum x_column value of the filtered DataFrame

    # Update layout
    fig.update_layout(
        xaxis_title='Time [sec]',
        yaxis_title=y_axis_name,
        title=y_column + ' Subject ' + numeric_part,
        showlegend=True
    )

    # Show the plot
    fig.show()
    fig.write_html(f'{y_column}_Subject_{numeric_part}.html')


אתה צריך לשים רקאת הכתובת של התיקייה שאליה ישמרו כל הדברים
,
את הכתובת של קובץ טקסט שאנחנו מוציאים שמכיל את הזמנים והדגימות

ואת הקובץ מטלב של הטובי 

In [18]:
output_directory = r"C:\Users\hadar.yehuda\Desktop\אנדריי\Name4"
file_path_txt = r"C:\Users\hadar.yehuda\Desktop\אנדריי\Name4\Name4.txt"
mat = loadmat(r"C:\Users\hadar.yehuda\Desktop\אנדריי\Name4\Name4_run1.mat")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hadar.yehuda\\Desktop\\אנדריי\\Name4\\Name4_run1.mat'

In [ ]:
target_keyword = "_converted"
file_list = os.listdir(output_directory)
txt_files_with_keyword = [file for file in file_list if target_keyword in file and file.endswith('.txt')]
matching_file_paths = [os.path.join(output_directory, file) for file in txt_files_with_keyword]
file_path_plux =matching_file_paths[0]
file_path_plux

In [ ]:
path_parts = output_directory.split("\\")
last_part = path_parts[-1]
numeric_part = ''.join(filter(str.isdigit, last_part))

In [ ]:
with open(file_path_txt, "r") as file:
    lines = file.readlines()

filtered_lines = [line.strip() for line in lines if "Sample movie start" in line or "Sample movie end" in line]
filtered_lines_with_date = [line for line in filtered_lines if re.search(r'\d{2}-\w{3}-\d{4}', line)]

start_numbers = []
end_numbers = []
start_times = []
end_times = []

for i in range(len(lines)):
    line = lines[i].strip()

    if "The first BioSignals" in line:
        next_line = lines[i + 2]  # Get the line containing the number
        number = float(next_line.strip().split()[0])  # Extract the number
        start_numbers.append(number)

        match = re.search(r'\d{2}:\d{2}:\d{2}', lines[i - 1])
        if match:
            start_times.append(match.group())

    elif "The last BioSignals" in line:
        next_line = lines[i + 2]  # Get the line containing the number
        number = float(next_line.strip().split()[0])  # Extract the number
        end_numbers.append(number)

        match = re.search(r'\d{2}:\d{2}:\d{2}', lines[i - 1])
        if match:
            end_times.append(match.group())

print("start_numbers =", start_numbers)
print("end_numbers =", end_numbers)
print("start_times =", start_times)
print("end_times =", end_times)

In [ ]:
start_systemTimeStamp = []
end_systemTimeStamp = []
start_times_t = []
end_times_t = []

for line in lines:
    line = line.strip()

    if "*Start* time :" in line:
        number = re.search(r': (\d+)', line)
        if number:
            start_systemTimeStamp.append(number.group(1))

        match = re.search(r'\d{2}:\d{2}:\d{2}', line)
        if match:
            start_times_t.append(match.group())

    if "*End* time :" in line:
        number = re.search(r': (\d+)', line)
        if number:
            end_systemTimeStamp.append(number.group(1))

        match = re.search(r'\d{2}:\d{2}:\d{2}', line)
        if match:
            end_times_t.append(match.group())

print("Start System TimeStamp =", start_systemTimeStamp)
print("End System TimeStamp =", end_systemTimeStamp)
print("start_times =", start_times_t)
print("end_times =", end_times_t)

In [ ]:
data_left = mat['dat']['data']['gaze']['left']['pupil']['diameter']
data_right = mat['dat']['data']['gaze']['right']['pupil']['diameter']

In [ ]:
DisplayArea_left_x = mat['dat']['data']['gaze']['left']['gazePoint']['onDisplayArea'][0]
DisplayArea_right_x = mat['dat']['data']['gaze']['right']['gazePoint']['onDisplayArea'][0]
DisplayArea_left_y = mat['dat']['data']['gaze']['left']['gazePoint']['onDisplayArea'][1]
DisplayArea_right_y = mat['dat']['data']['gaze']['right']['gazePoint']['onDisplayArea'][1]

In [ ]:
time_stamp = mat['dat']['data']['gaze']['systemTimeStamp']
df_t = pd.DataFrame({'time_stamp' : time_stamp,
                     'data_left':data_left ,'data_right':data_right,'DisplayArea_left_x':DisplayArea_left_x,
                     'DisplayArea_right_x':DisplayArea_right_x,'DisplayArea_left_y':DisplayArea_left_y,
                     'DisplayArea_right_y':DisplayArea_right_y})

In [ ]:
df_t = calculate_z_scores(df_t, 'data_left')
df_t = calculate_z_scores(df_t, 'data_right')

In [ ]:
lines_to_skip = 3  
df = pd.read_csv(file_path_plux, delimiter="\t", skiprows=range(lines_to_skip), header=None)
df = df.iloc[:, :5]
df.columns = ["sample_num", "Column2", "PZT", "BVP" ,"EDA"]
df = calculate_z_scores(df, 'PZT')
df = calculate_z_scores(df, 'BVP')
df = calculate_z_scores(df, 'EDA')

In [ ]:
df_t.to_csv(os.path.join(output_directory, 'reference_data.csv'), index=False)
df.to_csv(os.path.join(output_directory, 'data.csv'), index=False)

כאן אתה צריך ללכת לקוד של המטלב שנקרא resample  
לשנות את כתובות של התיקייה במטלב ולהריץ  
data_file_path = "C:\Users\hadar.yehuda\Desktop\אנדריי\Name4\data.csv";
reference_file_path = "C:\Users\hadar.yehuda\Desktop\אנדריי\Name4\reference_data.csv";
אחרי שזה רץ אתה ממשיך את הקוד 

In [ ]:
csv_path = os.path.join(output_directory, "sorted_resampled_data.csv")
resampled_df = pd.read_csv(csv_path)
sorted_resampled_df = resampled_df.sort_values(by='sample_num')

In [ ]:
merged_df = pd.merge(sorted_resampled_df, df_t, left_index=True, right_index=True, how='inner')
merged_df.to_csv(os.path.join(output_directory, 'all_data.csv'), index=False)

In [ ]:
divided_dfs = []
resampled_df = sorted_resampled_df
for i in range(len(start_numbers)):
    start = start_numbers[i]
    end = end_numbers[i]
    subset_df = resampled_df[(resampled_df["sample_num"] >= start) & (resampled_df["sample_num"] <= end)]
    subset_df = subset_df.reset_index(drop=True)  # Reset the index
    divided_dfs.append(subset_df)

#for i, df in enumerate(divided_dfs):
#    print(f"Subset {i+1}:")
#    print(df)
#    print()

In [ ]:
# Iterate over the divided DataFrames
for i, df in enumerate(divided_dfs):
    start_time = pd.to_datetime(start_times[i], format="%H:%M:%S")  # Convert start time to datetime object
    end_time = pd.to_datetime(end_times[i], format="%H:%M:%S")  # Convert end time to datetime object
    
    # Calculate the time interval between start and end time
    time_interval = (end_time - start_time) / (len(df) - 1)
    
    # Calculate the time for each sample based on the violation count
    sample_times = [start_time + idx * time_interval for idx in range(len(df))]
    
    # Calculate the time in seconds since the beginning of the interval
    time_deltas = [(t - start_time).total_seconds() for t in sample_times]    
    df['time_N'] = time_deltas
    
    # Print the modified DataFrame
    print(f"Subset {i+1}:")
    print(df)
    print()


In [ ]:
os.makedirs(output_directory, exist_ok=True)

# Iterate over the divided DataFrames
for i, df in enumerate(divided_dfs):
    # Construct the CSV file name using the loop index
    csv_file_name = f"movie_{i+1}_opensignals.csv"
    
    # Save the DataFrame to a CSV file
    df.to_csv(os.path.join(output_directory, csv_file_name), index=False)

    print(f"Subset {i+1} saved to {csv_file_name}")

In [ ]:
plot_divided_dfs(divided_dfs, 'time_N', 'z_score_EDA',numeric_part , 'EDA[uS]')

In [ ]:
plot_divided_dfs(divided_dfs, 'time_N', 'z_score_BVP',numeric_part , 'BVP[µV]',-1.5,1.5)

In [ ]:
plot_divided_dfs(divided_dfs, 'time_N', 'z_score_PZT', numeric_part, 'PZT[v]')

In [ ]:
divided_dfs_t = []

for i in range(len(start_systemTimeStamp)):
    start = int(start_systemTimeStamp[i])
    end = int(end_systemTimeStamp[i])
    subset_df = df_t[(df_t["time_stamp"] >= start) & (df_t["time_stamp"] <= end)]
    subset_df = subset_df.reset_index(drop=True)  # Reset the index
    divided_dfs_t.append(subset_df)

for i, df in enumerate(divided_dfs_t):
    print(f"Subset {i+1}:")
    print(df)
    print()

In [ ]:
# Iterate over the divided DataFrames
for i, df in enumerate(divided_dfs_t):
    start_time = pd.to_datetime(start_times[i], format="%H:%M:%S")  # Convert start time to datetime object
    end_time = pd.to_datetime(end_times[i], format="%H:%M:%S")  # Convert end time to datetime object
    
    # Calculate the time interval between start and end time
    time_interval = (end_time - start_time) / (len(df) - 1)
    # Calculate the time for each sample based on the violation count
    sample_times = [start_time + idx * time_interval for idx in range(len(df))]
    
    # Add the time column to the DataFrame and remove the date portion
    time_deltas = [(t - start_time).total_seconds() for t in sample_times]    
    df['time_N'] = time_deltas

    df['pupil Diameter_average'] = df[['data_left', 'data_right']].mean(axis=1)
    df['pupil Diameter'] = df[['z_score_data_left', 'z_score_data_right']].mean(axis=1)
    df['average_DisplayArea_y'] = df[['DisplayArea_left_y', 'DisplayArea_right_y']].mean(axis=1)
    df['average_DisplayArea_x'] = df[['DisplayArea_left_x', 'DisplayArea_right_x']].mean(axis=1)  
    
    # Print the modified DataFrame
    print(f"Subset {i+1}:")
    print(df)
    print()

In [ ]:
plot_divided_dfs(divided_dfs_t, 'time_N', 'pupil Diameter',numeric_part, 'pupil Diameter [pixels]')

In [ ]:
os.makedirs(output_directory, exist_ok=True)

# Iterate over the divided DataFrames
for i, df in enumerate(divided_dfs_t):
    # Construct the CSV file name using the loop index
    csv_file_name = f"movie_{i+1}_processed.csv"
    
    # Save the DataFrame to a CSV file
    df.to_csv(os.path.join(output_directory, csv_file_name), index=False)

    print(f"Subset {i+1} saved to {csv_file_name}")